In [45]:
date_spot = "2024.05.09"

In [46]:
import json

import pandas as pd

song_info = pd.read_csv("songList_temp.csv")

def get_user_set_unit(rank):
    user_set = set()
    for item in rank:
        for instance in rank[item]:
            user_set.add(instance["username"] + " " + instance["userid"])

    return user_set

#전체 랭킹 데이터에 포함된 user명의 set을 구함 user_name + " " + user_id
def get_user_set(ranks):
    user_set = set()
    for rank in ranks:
        user_set = user_set.union(get_user_set_unit(rank))
    return user_set

#중복된 유저명이 있는지 확인
def get_duplicate_usernames_dict(user_set):
    user_dict = {}
    for user in user_set:
        username = user.split()[0]
        if username not in user_dict:
            user_dict[username] = [user.split()[1]]
        else:
            user_dict[username].append(user.split()[1])

    filtered_dict = {k: v for k, v in user_dict.items() if len(v) > 1}
    return filtered_dict

def find_user(data, username, user_id = None):
    if user_id != None:
        if type(user_id) == int:
            user_id = "#"+ str(user_id)
    #print(user_id)
    instances = []
    for item in data:
        for instance in data[item]:
            if instance["username"] == username and (user_id == None or instance["userid"] == user_id):
                instances.append(instance)
    return instances

#Username/user_id가 일치하는 랭킹 정보를 수집함
def find_user_all(ranks, username, user_id = None):
    ans = []
    for instance in ranks:
        instances = find_user(instance, username, user_id)
        ans.append(instances)
    flattened_ans = [val for sublist in ans for val in sublist]
    return flattened_ans

#모든 유저의 랭킹 정보를 수집함 모든 유저들을 포함하는 집합을 만들고 그 유저들의 랭킹 정보를 수집함
def collect_all_userdata(ranks):
    user_set = get_user_set(ranks)
    user_dict = {}
    for user in user_set:
        username = user.split()[0]
        user_id = user.split()[1]
        instances = find_user_all(ranks, username, user_id)
        user_dict[user] = instances
    return user_dict


def get_all_rankdata(date_spot):
    filepath = "in100RankData/" + date_spot
    ranks = ["rankdata_27over", "rankdata_26", "rankdata_25", "rankdata_24","rankdata_23","rankdata_22","rankdata_21","rankdata_20"]
    all_rankdata = []
    for rank in ranks:
        current_rank = json.load(open(filepath + "/" + rank + ".json"))
        all_rankdata.append(current_rank)
    return all_rankdata

rank_all = get_all_rankdata(date_spot)
total_rank_raw = collect_all_userdata(rank_all)

In [47]:
def get_song_info(song_csv, song_name):
    for i in range(len(song_csv["song"])):
        if song_csv["song"][i].lower() == song_name.lower():
            song_type = song_csv["song_type"][i]
            song_version = song_csv["version"][i]
            return song_type, song_version
    print("Song not found in the songlist")
    return "N/A", "N/A"

def get_all_song_titles(level):
    set_song_titles = set()
    song_titles = list(level.keys())
    for item in song_titles:
        set_song_titles.add(item[:-9])
    return set_song_titles


all_songs = set()
for item in rank_all:
    all_songs = all_songs.union(get_all_song_titles(item))

print(len(all_songs))
for item in all_songs:
    if item not in song_info["song"].values:
        print(item)

388


In [48]:
def DA_0():
    return [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
def DA_l():
    return [[[],[],[]],[[],[],[]],[[],[],[]],[[],[],[]]]

def songtype_to_int(songtype):
    if songtype == "Arcade":
        return 0
    elif songtype == "Remix":
        return 1
    elif songtype == "Full Song":
        return 2
    elif songtype == "Short Cut":
        return 3
    else:
        return -1

def version_to_int(version):
    if version == "PHOENIX":
        return 0
    elif version == "XX":
        return 1
    else:
        return 2

def restrict_level_range(levels):
    if levels[1] > 28: levels[1] = 28
    if levels[0] < 20:
        levels[0] = 20
    return levels

def add_dict(a, b):
    for item in b:
        if item in a:
            a[item] += b[item]
        else:
            a[item] = b[item]
    return a

def get_song_version_int_list(songtype, version):
    songtypeint = []
    for item in songtype:
        songtypeint.append(songtype_to_int(item))
    versionint = []
    for item in version:
        versionint.append(version_to_int(item))
    return songtypeint, versionint

version_all = ["PHOENIX", "XX", "OLD"]
songtype_all = ["Arcade", "Remix", "Full Song", "Short Cut"]

class UserRank:
    
    def __init__(self, key, rank_dict_list):
        self.key = key
        username, userID = key.split()
        self.username = username
        self.userID = userID
        self.single_data = {key: DA_l() for key in range(20, 27)}
        self.single_counts = {key: DA_0() for key in range(20, 27)}
        self.double_data = {key: DA_l() for key in range(20, 29)}
        self.double_counts = {key: DA_0() for key in range(20, 29)}
        self.read_rank_dict(rank_dict_list)

            
    def read_rank_dict(self, rank_dict_list):
        for item in rank_dict_list:
            songname = item["song"]
            song_type, song_version = get_song_info(song_info, songname)
            item["type"] = song_type
            item["version"] = song_version
            level = item["level"]
            mode = item["mode"]
            typeint, versionint = songtype_to_int(item["type"]), version_to_int(item["version"])

            if mode == "Single":
                self.single_data[level][typeint][versionint].append(item)
                self.single_counts[level][typeint][versionint] += 1
            elif mode == "Double":
                self.double_data[level][typeint][versionint].append(item)
                self.double_counts[level][typeint][versionint] += 1

    def count_total(self):
        count = 0
        for level in range(20, 27):
            for songtype in range(4):
                for version in range(3):
                    count += self.single_counts[level][songtype][version]
        for level in range(20, 29):
            for songtype in range(4):
                for version in range(3):
                    count += self.double_counts[level][songtype][version]
        return count

        
    def to_dict(self):
        return {
            "key": self.key,
            "username": self.username,
            "userID": self.userID,
            "single_data": self.single_data,
            "single_counts": self.single_counts,
            "double_data": self.double_data,
            "double_counts": self.double_counts
        }
   
    def __str__(self):
        return f"{self.username} ({self.userID}): {self.count_total()}"

users = {}

for key in total_rank_raw:
    users[key] = UserRank(key, total_rank_raw[key])
users_dict = {key: users[key].to_dict() for key in users}

# Dump the new dictionary to a JSON file
with open(f"in100RankData/{date_spot}/user_data.json", "w", encoding="utf-8") as file:
    json.dump(users_dict, file, ensure_ascii=False)


In [49]:
#EXECUTE ALL ABOVE CELLS